## Install Packages

In [1]:
try:
    import pymongo
except:
    !pip install pymongo

try:
    import folium
except:
    !pip install folium

     |████████████████████████████████| 95 kB 716 kB/s            
     |████████████████████████████████| 15.7 MB 1.4 MB/s            


## Conncet to MongoDB

In [2]:
from pymongo import MongoClient
client = MongoClient('mongo:27017', 
                        username='root',
                         password='example')

# client = MongoClient('localhost:27017', username='root',password='example') # In case access from another instance  

## list of all databases

In [3]:
for db in client.list_databases():
    print(db)

{'name': 'admin', 'sizeOnDisk': 102400, 'empty': False}
{'name': 'config', 'sizeOnDisk': 12288, 'empty': False}
{'name': 'local', 'sizeOnDisk': 73728, 'empty': False}
{'name': 'sarah', 'sizeOnDisk': 40960, 'empty': False}


In [4]:
for db in client.list_database_names():
    print(db)

admin
config
local
sarah


## Create DataBase

In [5]:
db = client['covid19_db2020']

## Create Collection 

In [6]:
cases = db['cases_col_2020']


## Read json file 

In [7]:
!ls data_json |wc -l

35502


In [8]:
!ls data_json | head -n 10

0.json
10000.json
10001.json
10002.json
10003.json
10004.json
10005.json
10006.json
10007.json
10008.json
ls: write error: Broken pipe


In [9]:
import json 
with open('data_json/1.json', 'r') as f:
    file_dict=json.loads(json.load(f))

In [10]:
file_dict

{'Date': '3/8/2020',
 'Country_Region': 'India',
 'Province_State': None,
 'Difference': 0,
 'Prep_Flow_Runtime': '3/24/2020 9:39:03 AM',
 'Latest_Date': '3/23/2020',
 'Case_Type': 'Deaths',
 'Cases': 0,
 'Lat': 21.0,
 'Long': 78.0,
 'Index': 1}

In [11]:
!ls data_json | wc -l

35502


# Push documents to collection  

In [12]:
result = cases.insert_one(file_dict)

# Push all files 

In [13]:
import glob

List=glob.glob('data_json/*.json')

In [14]:
List[0:10]

['data_json/0.json',
 'data_json/1.json',
 'data_json/10.json',
 'data_json/100.json',
 'data_json/1000.json',
 'data_json/10000.json',
 'data_json/10001.json',
 'data_json/10002.json',
 'data_json/10003.json',
 'data_json/10004.json']

In [15]:
len(List)

35502

In [16]:
from tqdm import tqdm
for file in tqdm(List[1:1001]):
    with open(file, 'r') as f:
        file_dict=json.loads(json.load(f))
        
    result = cases.insert_one(file_dict)
    

100%|██████████| 1000/1000 [00:05<00:00, 168.74it/s]


## Return all the documents in a collection in mongodb

In [17]:
cursor = cases.find({})
for document in cursor:
    print(document)

{'_id': ObjectId('61a74ef4874b7858ccc45d9c'), 'Date': '3/8/2020', 'Country_Region': 'India', 'Province_State': None, 'Difference': 0, 'Prep_Flow_Runtime': '3/24/2020 9:39:03 AM', 'Latest_Date': '3/23/2020', 'Case_Type': 'Deaths', 'Cases': 0, 'Lat': 21.0, 'Long': 78.0, 'Index': 1}
{'_id': ObjectId('61a74ef9874b7858ccc45d9d'), 'Date': '3/8/2020', 'Country_Region': 'India', 'Province_State': None, 'Difference': 0, 'Prep_Flow_Runtime': '3/24/2020 9:39:03 AM', 'Latest_Date': '3/23/2020', 'Case_Type': 'Deaths', 'Cases': 0, 'Lat': 21.0, 'Long': 78.0, 'Index': 1}
{'_id': ObjectId('61a74ef9874b7858ccc45d9e'), 'Date': '3/20/2020', 'Country_Region': 'India', 'Province_State': None, 'Difference': 1, 'Prep_Flow_Runtime': '3/24/2020 9:39:03 AM', 'Latest_Date': '3/23/2020', 'Case_Type': 'Deaths', 'Cases': 5, 'Lat': 21.0, 'Long': 78.0, 'Index': 10}
{'_id': ObjectId('61a74ef9874b7858ccc45d9f'), 'Date': '2/20/2020', 'Country_Region': 'India', 'Province_State': None, 'Difference': 0, 'Prep_Flow_Runtime':

In [18]:
cursor = cases.find({})
Locations=[]
Name=[]
for document in tqdm(cursor):
    #print(document['Lat'],document['Long'],document['Case_Type'])
    Locations.append([document['Lat'],document['Long']])
    Name.append(document['Case_Type'])

1001it [00:00, 149359.60it/s]


In [19]:
import folium
map = folium.Map(location=[38.9, -77.05], zoom_start=8)
folium.Marker([38.9, -77.05], 
                  popup='PSUT is the best'
                  ).add_to(map)
folium.Marker([38.9, -77.2], 
                  popup='PSUT is the best always'
                  ).add_to(map)
map

In [20]:
import folium
map = folium.Map(location=[38.9, -77.05], zoom_start=2)
for point in range(0, len(Name)):
    folium.Marker(Locations[point], 
                  popup=Name[point]
                  ).add_to(map)
map

In [21]:
import folium
map = folium.Map(location=[38.9, -77.05],tiles="Stamen Terrain", zoom_start=2)
for point in range(0, len(Name)):
    folium.Marker(Locations[point], 
                  popup=Name[point]
                  ).add_to(map)
map



In [22]:


import folium
map = folium.Map(location=[38.9, -77.05],tiles="Stamen Toner", zoom_start=2)
for point in range(0, len(Name)):
    folium.Marker(Locations[point], 
                  popup=Name[point]
                  ).add_to(map)
map


In [23]:
map.save('CovideCases.html')

In [24]:
!du -sh data_json

0	data_json
